In [2]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import chi2_contingency, beta
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('/Users/yoo/Google Drive/내 드라이브/bayesian-testing-main/ab_data.csv')
print(df.shape)

df.head(10)

(294478, 5)


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [5]:
df.sample(2)

,user_id,timestamp,group,landing_page,converted
145374,656499,2017-01-19 22:46:03.164360,control,old_page,0
80423,743991,2017-01-22 00:55:07.680207,treatment,new_page,0


In [15]:
start_time = datetime.datetime.strptime(df['timestamp'].min(), '%Y-%m-%d %H:%M:%S.%f')
end_time = datetime.datetime.strptime(df['timestamp'].max(), '%Y-%m-%d %H:%M:%S.%f')
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiments: {df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare: {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control: {round(df[df['group'] == 'control'].shape[0] * 100 / df.shape[0],2)}%")

Number of unique users in experiments: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 49.99%


In [16]:
sample = df[df['user_id'].isin([746755,722274])]
sample

,user_id,timestamp,group,landing_page,converted
29073,746755,2017-01-11 01:28:57.083669,control,new_page,1
105487,722274,2017-01-19 01:46:53.093257,control,old_page,0
262554,722274,2017-01-09 21:21:23.638444,control,new_page,0
286566,746755,2017-01-05 03:40:08.457451,control,old_page,0


In [17]:
first_conversion = sample.groupby('user_id')['timestamp'].min().to_frame().reset_index()
sample = sample.merge(first_conversion, on=['user_id', 'timestamp'])
sample

,user_id,timestamp,group,landing_page,converted
0,722274,2017-01-09 21:21:23.638444,control,new_page,0
1,746755,2017-01-05 03:40:08.457451,control,old_page,0


In [20]:
# 3894(1.34%) 유저가 A/B 둘 다 노출됨. 해당 데이터를 삭제해도 괜찮을 듯 함.

counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

In [24]:
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df = df.merge(valid_users, on=['user_id'])

print(df.shape)
df.head(10)

(286690, 5)


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [25]:
df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').isocalendar()[1])
df.sample(random_state=9095)

,user_id,timestamp,group,landing_page,converted,week
192845,798948,2017-01-10 07:47:51.948129,treatment,new_page,0,2


In [27]:
df['week'].value_counts().sort_index()

1    83745
2    91380
3    91056
4    20509
Name: week, dtype: int64

### Frequentist Approach

In [33]:
for i in df['week'].unique():
    print(f"{i} Weekend")
    num_week = i
    experiment_data = df[df['week'] <= num_week]
    control = experiment_data[experiment_data['group'] == 'control']
    treatment = experiment_data[experiment_data['group'] == 'treatment']

    control_conversion_perc = round(control['converted'].sum() * 100 / control['converted'].count(), 3)
    treatment_conversion_perc = round(treatment['converted'].sum() * 100 / treatment['converted'].count(), 3)

    lift = round(treatment_conversion_perc - control_conversion_perc, 3)

    print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
    print(f"Control Conversion Rate: {control_conversion_perc}%")
    print(f"Lift = {lift}%")

3 Weekend
Treatment Conversion Rate: 11.859%
Control Conversion Rate: 11.992%
Lift = -0.133%
2 Weekend
Treatment Conversion Rate: 11.802%
Control Conversion Rate: 11.922%
Lift = -0.12%
1 Weekend
Treatment Conversion Rate: 11.784%
Control Conversion Rate: 11.919%
Lift = -0.135%
4 Weekend
Treatment Conversion Rate: 11.873%
Control Conversion Rate: 12.017%
Lift = -0.144%


In [34]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

contingency_table = np.array([[control_converted, control_non_converted],
                              [treatment_converted, treatment_non_converted]])

In [35]:
contingency_table

array([[ 17220, 126073],
       [ 17025, 126372]])

In [40]:
chi, p_value, _, _= chi2_contingency(contingency_table, correction=False)

In [43]:
print(f"chi: {chi}, p-value: {p_value}")

chi: 1.426794609399621, p-value: 0.23228827305833816


In [45]:
print(f"{round(p_value*100,2)}% Probability that a more extreme chi square than {chi} would have occured by chance")

23.23% Probability that a more extreme chi square than 1.426794609399621 would have occured by chance


In [46]:
print(f"We can't say this, We are {round(p_value*100,2)}% confident that our lift = {lift}%")

We can't say this, We are 23.23% confident that our lift = -0.144%


### Bayesian Approach

In [48]:
prior = df[(df['week'] == 1) & (df['group'] == 'control')]
print(prior.shape)

(41731, 6)


In [49]:
prior_means = []

for i in range(10000):
    prior_means.append(prior.sample(1000)['converted'].mean())
    
prior_means[:10]

[0.13, 0.128, 0.097, 0.117, 0.134, 0.132, 0.122, 0.138, 0.112, 0.126]

In [50]:
prior_alpha, prior_beta, _, _ = beta.fit(prior_means, floc=0, fscale=1)

In [59]:
for i in df['week'].unique():
    print(f"{i} Weeks")
    num_week = i
    experiment_data = df[(df['week'] > 1) & (df['week'] <= i)]
    control = experiment_data[experiment_data['group'] == 'control']
    treatment = experiment_data[experiment_data['group'] == 'treatment']
    
    control_conversion_perc = round(control['converted'].sum() * 100 / control['converted'].count(), 2)
    treatment_conversion_perc = round(treatment['converted'].sum() * 100 / treatment['converted'].count(), 2)
    
    lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc, 2)
    
    print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
    print(f"Control Conversion Rate: {control_conversion_perc}%")
    print(f"Lift = {lift}%")

3 Weeks
Treatment Conversion Rate: 11.89%
Control Conversion Rate: 12.03%
Lift = -0.01%
2 Weeks
Treatment Conversion Rate: 11.82%
Control Conversion Rate: 11.93%
Lift = -0.01%
1 Weeks
Treatment Conversion Rate: nan%
Control Conversion Rate: nan%
Lift = nan%
4 Weeks
Treatment Conversion Rate: 11.91%
Control Conversion Rate: 12.06%
Lift = -0.01%


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


In [69]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)

probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {round(probability * 100, 2)}%")

Probability that treatment > control: 16.3%


In [70]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()

print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}")
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.12056262260650974, Variance: 1.033533904228902e-06
Treatment Posterior: Mean: 0.11909378649071319, Variance: 1.0244348457751666e-06


In [71]:
lift_percentage = (treatment_samples - control_samples) / control_samples

print(f"Probability that we are seeing a 2% lift: {np.mean((100 * lift_percentage) > 2) * 100}%")

Probability that we are seeing a 2% lift: 0.3%
